In [1]:
import nltk

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('reuters')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\zhuko\AppData\Roaming\nltk_data...


True

In [2]:
from nltk.lm import MLE
from nltk.corpus import reuters
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize

In [3]:
# Подготовка данных
texts = reuters.sents()

In [53]:
texts

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]

In [54]:
# Преобразование текстов в последовательности n-gram
n = 3
train_data, padded_vocab = padded_everygram_pipeline(n, texts)

In [38]:
# Обучение модели MLE
model = MLE(n)
model.fit(train_data, padded_vocab)

In [55]:
context = ("the","stock")

In [56]:
print("Следующее слово:", model.generate(2, context, random_seed=42))

Следующее слово: ['of', '&']


In [57]:
# Получаем вероятности всех слов в словаре
#scores = {word: model.score(word, context) for word in model.vocab}
scores = {word: model.score(word, context) for word in model.vocab if word.isalpha()}

In [58]:
# Отсортируем слова по их вероятности и выберем топ-5
top_5 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]

In [59]:
# Выводим результаты
print("Топ-5 слов с максимальной вероятностью:")
for word, score in top_5:
    print(f"{word}: {score:.4f}")

Топ-5 слов с максимальной вероятностью:
of: 0.1088
for: 0.0881
market: 0.0829
split: 0.0518
to: 0.0415


In [39]:
from collections import defaultdict, Counter

In [40]:
# Создание частотного распределения для каждого контекста
context_word_freq = defaultdict(Counter)

for sentence in texts:
    for i in range(len(sentence) - 2):
        context = (sentence[i], sentence[i + 1])
        word = sentence[i + 2]
        context_word_freq[context][word] += 1

In [44]:
context = ("the", "stock")

In [45]:
# Получаем топ-5 слов для заданного контекста
top_words = context_word_freq[context].most_common(5)

In [46]:
# Выводим результат
print("Топ-5 слов для контекста", context, ":", top_words)

Топ-5 слов для контекста ('the', 'stock') : [('of', 21), ('for', 17), ('market', 16), ('split', 10), (',', 9)]


In [60]:
context_counts = model.context_counts(model.vocab.lookup(context))
sorted_words = sorted(context_counts.items(), key=lambda item: item[1], reverse=True)
top_5_words = [word for word, count in sorted_words[:5]]

In [61]:
print("Топ-5 следующих слов для заданного контекста:", top_5_words)

Топ-5 следующих слов для заданного контекста: ['of', 'for', 'market', 'split', ',']
